# DATA

In [52]:
#Importing relevant libraries
import bs4 as bs
import urllib.request 
import time
from datetime import datetime 
import pandas as pd
import json
import requests
import os
import folium

In [14]:
#Load data 
url = 'https://suedbayerische-immobilien.de/Mietpreise-Muenchen-Stadtteile'
df=pd.read_html(url, header=0)[0]
df.columns=['Neighbordhood','Rent per M2']
df

,Neighbordhood,Rent per M2
0,Altstadt – Lehel,1915
1,Schwabing,1876
2,Maxvorstadt – Universitätsviertel,1861
3,Ludwigsvorstadt – Isarvorstadt,1748
4,Maxvorstadt,1741
5,Schwabing-West,1731
6,Au – Haidhausen,1696
7,Altbogenhausen – Herzogpark,1691
8,Nymphenburg,1647
9,Schwanthalerhöhe,1605


In [21]:
df.info()
print ('---')
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 2 columns):
Neighbordhood    33 non-null object
Rent per M2      33 non-null int64
dtypes: int64(1), object(1)
memory usage: 608.0+ bytes
---


,Rent per M2
count,33.000000
mean,1510.606061
std,185.027353
min,1219.000000
25%,1375.000000
50%,1459.000000
75%,1647.000000
max,1915.000000


In [43]:
#Get Latitude and Longitude
geo_data = pd.read_excel("Geospatial_Coordinates.xls")
geo_data
df['Latitude']=geo_data['Latitude'].values
df['Longitude']=geo_data['Longitude'].values
df

,Neighbordhood,Rent per M2,Latitude,Longitude
0,Altstadt – Lehel,1915,48.139130,11.580220
1,Schwabing,1876,48.167450,11.570380
2,Maxvorstadt – Universitätsviertel,1861,48.147629,11.568050
3,Ludwigsvorstadt – Isarvorstadt,1748,48.132440,11.556090
4,Maxvorstadt,1741,48.151092,11.562418
5,Schwabing-West,1731,48.168271,11.569873
6,Au – Haidhausen,1696,48.128753,11.590536
7,Altbogenhausen – Herzogpark,1691,48.149398,11.603792
8,Nymphenburg,1647,48.158257,11.503297
9,Schwanthalerhöhe,1605,48.134230,11.539034


In [46]:
center_lat=df.Latitude.mean()
center_long=df.Longitude.mean()

# create map of Munich using latitude and longitude values
map_to = folium.Map(location=[center_lat, center_long], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighbordhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  

map_to

In [47]:
CLIENT_ID = 'FUZFUOSOAV35IPWPTAYMF2HV3EEX3IQKUQ1JPPA1ZHYL4GMN' # your Foursquare ID
CLIENT_SECRET = 'Y2S3SBIZMUD3V2L4Z5EFKLH2EINWA2SOELDPTKVD0HRFM4O2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FUZFUOSOAV35IPWPTAYMF2HV3EEX3IQKUQ1JPPA1ZHYL4GMN
CLIENT_SECRET:Y2S3SBIZMUD3V2L4Z5EFKLH2EINWA2SOELDPTKVD0HRFM4O2


In [49]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
munich_venues = getNearbyVenues(names=df['Neighbordhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Altstadt – Lehel
Schwabing
Maxvorstadt – Universitätsviertel
Ludwigsvorstadt – Isarvorstadt
Maxvorstadt
Schwabing-West
Au – Haidhausen
Altbogenhausen – Herzogpark
Nymphenburg
Schwanthalerhöhe
Harlaching
Neuhausen
Sendling
Untergiesing
Alte Heide – Hirschau
Obergiesing
Thalkirchen – Obersendling – Forstenried – Fürstenried – Solln
Oberföhring – Englschalking
Milbertshofen
Moosach
Laim
Pasing – Obermenzing
Hadern
Trudering – Riem
Sendling – Westpark
Ramersdorf – Perlach
Berg am Laim
Freimann
Johanneskirchen – Daglfing
Am Hart
Allach – Untermenzing
Aubing – Lochhausen – Langwied
Feldmoching – Hasenbergl


In [54]:
print('There are {} single categories.'.format(len(munich_venues['Venue Category'].unique())))

There are 170 single categories.


In [55]:
munich_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allach – Untermenzing,5,5,5,5,5,5
Altbogenhausen – Herzogpark,19,19,19,19,19,19
Alte Heide – Hirschau,16,16,16,16,16,16
Altstadt – Lehel,100,100,100,100,100,100
Am Hart,4,4,4,4,4,4
Au – Haidhausen,41,41,41,41,41,41
Aubing – Lochhausen – Langwied,9,9,9,9,9,9
Berg am Laim,8,8,8,8,8,8
Feldmoching – Hasenbergl,12,12,12,12,12,12


In [58]:
munich_onehot = pd.get_dummies(munich_venues[['Venue Category']], prefix="", prefix_sep="")
munich_onehot.head()

,Accessories Store,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,...,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
munich_grouped = munich_onehot.groupby('Neighbordhood').mean().reset_index()
munich_grouped

KeyError: 'Neighbordhood'